In [3]:
from keras.applications.vgg16 import VGG16

# load model
vgg = VGG16(weights="imagenet",include_top=False,input_shape=(224,224,3))
# freeze
vgg.trainable = False
# summarize the model
vgg.summary()

2023-01-07 12:12:28.788974: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-07 12:12:30.204888: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-12.0/lib64:/usr/local/cuda/lib64:/usr/local/cuda-11.8/lib64:/usr/local/cuda-11.8/extras/CUPTI/lib64:/usr/local/cuda-12.0/extras/CUPTI/lib64
2023-01-07 12:12:30.205114: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or d

58889256/58889256 [==============================] - 12s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                             

In [4]:
import enum
import numpy as np
import cv2 
from matplotlib import pyplot as plt
from keras.layers import *
import os

class CatFeatures(enum.Enum):
    # below are the cat facial features (⊙o⊙)
    # eyes
    LEFT_EYE = 0
    RIGHT_EYE = 1
    # mouth
    MOUTH = 2
    # left ear
    LEFT_EAR_1 = 3
    LEFT_EAR_2 = 4
    LEFT_EAR_3 = 5
    # right ear
    RIGHT_EAR_1 = 6
    RIGHT_EAR_2 = 7
    RIGHT_EAR_3 = 8
    
def load_image(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    labels = load_labels(path)[1:]

    w,h = img.shape[:2]
    
    return img, labels , w , h
    
def load_labels(path):
    path = path + ".cat"
    
    with open(path,'r') as f:
        coordinates = f.readline()
        coordinates = str(coordinates).split(' ')[:-1]
    
    return list(map(int,coordinates))

def map_labels(labels):
    x = labels[0:18:2]
    y = labels[1:18:2]
    
    features ={
        CatFeatures.LEFT_EYE : (),
        CatFeatures.RIGHT_EYE : (),
        CatFeatures.MOUTH : (),
        CatFeatures.LEFT_EAR_1 : (),
        CatFeatures.LEFT_EAR_2 : (),
        CatFeatures.LEFT_EAR_3 : (),
        CatFeatures.RIGHT_EAR_1 : (),
        CatFeatures.RIGHT_EAR_2 : (),
        CatFeatures.RIGHT_EAR_3 : (),
              }
    for key,xpoint,ypoint in zip(features.keys(),x,y):
        features[key] = (xpoint,ypoint)
    
    return features
    
def init_dataset(path,preprocess=False):
    root_path = path
    images = []
    labels = []
    
    for root,_,files in os.walk(root_path):
        for file in files:
            if file.endswith('.jpg'):
                x,y,w,h = load_image(os.path.join(root,file))
                
                if preprocess:
                    # preprocess data here !
                    x = preprocess_image(x)
                    # preprocess labels
                    y = preprocess_labels(y,w,h)
                images.append(x)
                labels.append(y)
    
    images = np.asarray(images)
    labels = np.asarray(labels)
    
    return images, labels.reshape(-1,18)

def preprocess_image(image):
    x = image / 255.0
    x = cv2.resize(x,(224,224))
    x = np.asarray(x).astype('float32')
    return x

def preprocess_labels(labels,width,height):
    y = labels
    y[0:18:2] = list(map(lambda point: point / width, y[0:18:2])) # x
    y[1:18:2] = list(map(lambda point: point / height, y[1:18:2])) # y
    return y

def create_dense_layer(nodes):
    layer = [
        Dense(nodes,activation='relu'),
        BatchNormalization(),
        #Dropout(0.2)
    ]
    return layer

def create_regression_net():
    start_nodes = 256
    nb_layers = 4
    
    regression = Sequential()
    regression.add(BatchNormalization())
    
    for i in range(nb_layers):
        nodes = start_nodes / 2
        
        layers = create_dense_layer(nodes)
        
        for l in layers:
            regression.add(l)
        
        start_nodes = nodes
        
    regression.add(Dense(18,activation='sigmoid'))
    
    return regression

def build_network(feature_extractor_net):
    model = Sequential()
    model.add(feature_extractor_net)
    model.add(Flatten())
    model.add(create_regression_net())

    return model

def decode_labels(labels,width,heigth):
    labels[0:18:2] = labels[0:18:2] * width
    labels[1:18:2] = labels[1:18:2] * heigth
    return labels

def predict_image(path, model):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    (w, h) = img.shape[:2]
    
    # predictions
    image_preprocessed = preprocess_image(img)
    y = model.predict(np.expand_dims(image_preprocessed, axis=0)).flatten()
    y = decode_labels(y, w, h)
    show_cat(img,y)

def show_cat(image,labels):
    features = map_labels(labels)

    plt.imshow(image)
    #plt.scatter(labels[0:18:2],labels[1:18:2],c='r',)
    
    
    x,y = [],[]
    points = [CatFeatures.LEFT_EAR_1,
              CatFeatures.LEFT_EAR_2,
              CatFeatures.LEFT_EAR_3,
              CatFeatures.RIGHT_EAR_1,
              CatFeatures.RIGHT_EAR_2,
              CatFeatures.RIGHT_EAR_3,
              CatFeatures.MOUTH,
              CatFeatures.LEFT_EAR_1,
             ]
    
    for p in points:
        x.append(features[p][0])
        y.append(features[p][1])
    
    lines = plt.plot(x,y,marker='*')
    plt.setp(lines, color='c',)
    plt.show()

In [5]:
# preprocess data
train_paths = ['../archive/CAT_00/','../archive/CAT_01/']

x_train,y_train = np.empty((0,224,224,3)), np.empty((0,18))

for path in train_paths:
    (x,y) = init_dataset(path,preprocess=True)
    x_train = np.vstack((x_train,x))
    y_train = np.vstack((y_train,y))


In [7]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_train,y_train, test_size=0.33, random_state=42)

In [8]:
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

model = build_network(vgg)
    
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 sequential_1 (Sequential)   (None, 18)                3323874   
                                                                 
Total params: 18,038,562
Trainable params: 3,273,218
Non-trainable params: 14,765,344
_________________________________________________________________


In [9]:
model.compile(optimizer=Adam(learning_rate=0.001),loss='mse',metrics=['accuracy'])

In [10]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=150,batch_size=32)

2023-01-07 12:16:04.721359: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1340903424 exceeds 10% of free system memory.
2023-01-07 12:16:06.193304: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1340903424 exceeds 10% of free system memory.


Epoch 1/150


2023-01-07 12:16:11.620615: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8700
2023-01-07 12:16:13.001707: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-01-07 12:16:15.538128: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x56553a09ba20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-01-07 12:16:15.538153: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 2070 with Max-Q Design, Compute Capability 7.5
2023-01-07 12:16:15.568752: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-01-07 12:16:15.802244: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-01-07 12:16:15.850456: I tensorflow/compiler

70/70 [==============================] - ETA: 0s - loss: 0.0899 - accuracy: 0.0337

2023-01-07 12:16:28.002633: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 660516864 exceeds 10% of free system memory.
2023-01-07 12:16:28.749484: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 660516864 exceeds 10% of free system memory.


70/70 [==============================] - 27s 236ms/step - loss: 0.0899 - accuracy: 0.0337 - val_loss: 0.0677 - val_accuracy: 0.0602
Epoch 2/150
70/70 [==============================] - 11s 158ms/step - loss: 0.0549 - accuracy: 0.0552 - val_loss: 0.0425 - val_accuracy: 0.0930
Epoch 3/150
70/70 [==============================] - 11s 159ms/step - loss: 0.0349 - accuracy: 0.1419 - val_loss: 0.0278 - val_accuracy: 0.3108
Epoch 4/150
70/70 [==============================] - 11s 160ms/step - loss: 0.0215 - accuracy: 0.2919 - val_loss: 0.0209 - val_accuracy: 0.4403
Epoch 5/150
70/70 [==============================] - 43s 626ms/step - loss: 0.0140 - accuracy: 0.4086 - val_loss: 0.0178 - val_accuracy: 0.4722
Epoch 6/150
70/70 [==============================] - ETA: 0s - loss: 0.0106 - accuracy: 0.4589

: 

: 

In [ ]:
img,l,w,h = load_image('../input/cat-dataset/CAT_05/00001100_003.jpg')

# ground truth
figure = plt.subplot(1,2,1)
figure.set_title('Ground Truth')
plt.axis('off')
plt.imshow(img)
plt.scatter(l[0:18:2],l[1:18:2])

# predictions

image = preprocess_image(img)
y = model.predict(np.expand_dims(image,axis=0)).flatten()
y = decode_labels(y,w,h)

figure = plt.subplot(1,2,2)
figure.set_title('Predicted')
plt.axis('off')
plt.imshow(img)
plt.scatter(y[0:18:2],y[1:18:2])

# def test_image(path):
#     img,l,w,h = load_image('../input/cat-dataset/CAT_01/00000100_003.jpg')



In [ ]:
model.save('pred.h5')

In [ ]:
predict_image('../input/cat-dataset/CAT_00/00000006_006.jpg',model)

In [ ]:
predict_image('../input/cat-dataset/CAT_00/00000011_014.jpg',model)